In [1]:
import gensim
import sys
sys.path.append('../Training/')
from res import *

C:\Users\Dell\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [30]:
df=pd.read_csv('../Data/cleanTweets.csv',na_values={'None','NONE'})

df=df.replace(r'^\s*$', np.nan, regex=True)
df=df.fillna('')

In [24]:
#loading SVM model
sl='../Training/learnWV8_1.sav' 
Llearn_obj=pickle.load(open(sl,'rb'))

#loading Word2Vec model
str_='../Training/w2v1_9_ESW5_C_swmod3_s_w2'
word2vec_path = str_+".bin"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

C:\Users\Dell\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [33]:
#This cell computes sentiment class for each tweet and corrosponding confidence score
import ast

def conf(arg):
    return max(arg[0],arg[1],arg[2])

df['tokens']=df.apply(lambda arg: ast.literal_eval(arg['tokens']),axis=1)

embed = get_word2vec_embeddings(word2vec, df)

df['pred']=Llearn_obj.predict(embed) #predicting sentiment class 
df['conf']=list(map(conf,Llearn_obj.predict_proba(embed))) #computing proability corrosponding to most confident prediction

In [35]:
df.to_csv('cleanTweets_pred_prob.csv',index=None,header=True) 